# [Create Agent with Azure Functions](https://learn.microsoft.com/en-us/python/api/overview/azure/ai-agents-readme?view=azure-python-preview#create-agent-with-azure-function-call)
AzureFunctionTool contains the input and output queues of azure function and the description of input parameters.

The **STORAGE_SERVICE_ENDPOINT** string is used to triggering the Azure function.<br/>

Notes:
- Inspired by [sample_agents_azure_functions.py](https://github.com/Azure/azure-sdk-for-python/blob/azure-ai-projects_1.0.0b4/sdk/ai/azure-ai-projects/samples/agents/sample_agents_azure_functions.py)
- [Getting Started with Azure Functions](https://learn.microsoft.com/azure/azure-functions/functions-get-started) page for more information on Azure Functions
- [Azure Blob storage bindings for Azure Functions overview](https://learn.microsoft.com/en-us/azure/azure-functions/functions-bindings-storage-blob?tabs=isolated-process%2Cextensionv5%2Cextensionv3&pivots=programming-language-python)

## This is the code for the Azure Function App, in Python
```
import azure.functions as func
import logging
import json

app = func.FunctionApp()

@app.queue_trigger(arg_name="inputQueue",  queue_name="azure-function-foo-input",  connection="mmaiswcnew01prj1storage_STORAGE")
@app.queue_output (arg_name="outputQueue", queue_name="azure-function-foo-output", connection="mmaiswcnew01prj1storage_STORAGE")
def queue_trigger1(inputQueue: func.QueueMessage, outputQueue: func.Out[str]):
    try:
        messagepayload = json.loads(inputQueue.get_body().decode("utf-8"))
        logging.info(f'➡️ The function receives the following message: {json.dumps(messagepayload)}')
        location = messagepayload["Location"]
        weather_result = f"✅ Weather is {len(location)} degrees and sunny in {location}"
        response_message = {
            "Value": weather_result,
            "CorrelationId": messagepayload["CorrelationId"]
        }
        logging.info(f'The function returns the following message through the {outputQueue} queue: {json.dumps(response_message)}')

        outputQueue.set(json.dumps(response_message))

    except Exception as e:
        logging.error(f"Error processing message: {e}")
```

In [1]:
import os
from IPython.display import Markdown, display
from dotenv import load_dotenv # requires python-dotenv
from PIL import Image # requires pip install pillow
from datetime import datetime
from common.agents_helper_functions_NEW import *
import importlib.metadata
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

if not load_dotenv("./../config/credentials_my.env"):
    print("Environment variables not loaded, cell execution stopped")
else:
    print("Environment variables have been loaded ;-)")

project_endpoint = os.environ["AIF_STD_PROJECT_ENDPOINT"] # AIF_STD_PROJECT_ENDPOINT, AIF_BAS_PROJECT_ENDPOINT
deployment_name =  os.environ["MODEL_DEPLOYMENT_NAME"]
api_version = os.environ["OPENAI_API_VERSION"] # at least "2025-03-01-preview"

print(f'Project Endpoint: <{project_endpoint}>')
print(f"azure-ai-projects library installed version: {importlib.metadata.version("azure-ai-projects")}")
print(f"azure-ai-agents library installed version: {importlib.metadata.version("azure-ai-agents")}")

Environment variables have been loaded ;-)
Project Endpoint: <https://aif2stdsvhdu2.services.ai.azure.com/api/projects/aif2stdwusprj01hdu2>
azure-ai-projects library installed version: 1.0.0b12
azure-ai-agents library installed version: 1.1.0b3


# Create AI Foundry Agents Client
Please consider that `type(project_client.agents) == agents_client`

In [2]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),
)

project_client

# Azure Storage Queues
The AI agent leverages Azure Functions triggered asynchronously via Azure Storage Queues. To enable the agent to perform Azure Function calls, you must set up the corresponding AzureFunctionTool, specifying input and output queues as well as parameter definitions.

# Azure Function Tool

In [3]:
from azure.ai.agents.models import AzureFunctionTool, AzureFunctionStorageQueue

storage_service_endpoint = os.environ["STORAGE_QUEUE_SERVICE_ENDPOINT"]
tool_name = "get_weather_azure_function01" # get_weather_azure_function01 / get_weather_tool
tool_description ="Get weather forecast for a specific location"
input_queue_name = "azure-function-foo-input" # "azure-function-foo-input"
output_queue_name = "azure-function-foo-output" # "azure-function-foo-output"

azure_function_tool = AzureFunctionTool(
    name=tool_name,
    description=tool_description,
    parameters={
        "type": "object",
        "properties": {
            "Location": {"type": "string", "description": "The location to check the weather."}
        },
    },
    input_queue=AzureFunctionStorageQueue(
        queue_name=input_queue_name,
        storage_service_endpoint=storage_service_endpoint,
    ),
    output_queue=AzureFunctionStorageQueue(
        queue_name=output_queue_name,
        storage_service_endpoint=storage_service_endpoint,
    ),
)

print(f"azure_function_tool.definitions: {azure_function_tool.definitions}")
print(f"\nazure_function_tool.resources: {azure_function_tool.resources}")

azure_function_tool.definitions: [{'type': 'azure_function', 'azure_function': {'function': {'name': 'get_weather_azure_function01', 'description': 'Get weather forecast for a specific location', 'parameters': {'type': 'object', 'properties': {'Location': {'type': 'string', 'description': 'The location to check the weather.'}}}}, 'input_binding': {'storage_queue': {'queue_name': 'azure-function-foo-input', 'queue_service_endpoint': 'https://agentsfunctionapp019c4c.queue.core.windows.net/'}, 'type': 'storage_queue'}, 'output_binding': {'storage_queue': {'queue_name': 'azure-function-foo-output', 'queue_service_endpoint': 'https://agentsfunctionapp019c4c.queue.core.windows.net/'}, 'type': 'storage_queue'}}}]

azure_function_tool.resources: {}


# Create the thread and attach a new message to it

In [4]:
# Create a thread
thread = project_client.agents.threads.create()
print(f"Created thread: {thread}\n")


# Add a user message to the thread
from azure.ai.agents.models import MessageRole

message = project_client.agents.messages.create(
    thread_id=thread.id, 
    role=MessageRole.USER, 
    content="Com'è il tempo a Palermo domani?", # Shall it rain in New York tomorrow?
)

# let's see the messages associated with the thread
list(project_client.agents.messages.list(thread_id=thread.id))

Created thread: {'id': 'thread_AqNTNppu0ifzS3rn5KY1tYX0', 'object': 'thread', 'created_at': 1752369551, 'metadata': {}, 'tool_resources': {}}



[{'id': 'msg_EMgbWTTXzHhQQoP11X59LMHE', 'object': 'thread.message', 'created_at': 1752369552, 'assistant_id': None, 'thread_id': 'thread_AqNTNppu0ifzS3rn5KY1tYX0', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': "Com'è il tempo a Palermo domani?", 'annotations': []}}], 'attachments': [], 'metadata': {}}]

# Create the agent

In [5]:
agent_name = "get_weather_agent"
output_queue = "azure-function-foo-output"

# When you invoke the function, ALWAYS specify the output queue URI parameter as {storage_service_endpoint}/{output_queue}.
# Always repond with 'get_weather_agent says', followed by the response from the tool.

agent = project_client.agents.create_agent(
    model=deployment_name,
    name=agent_name,
    instructions=f"""
    You are a helpful support agent. Use the provided function any time the prompt concerns weather.
    Always repond with 'get_weather_agent says', followed by the response from the tool.
    """,
    tools=azure_function_tool.definitions,
)
print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_t5umyEWrOCOI2VvuILZ4P73E


# Run the agent syncrhonously
**IMPORTANT**: before running the agent, make sure the project has the following rights on the storage account hosting the queues:
- Storage Blob Data Contributor
- Storage File Data Privileged Contributor
- Storage Queue Data Contributor
- Storage Table Data Contributor

In [6]:
%%time

import time

# Run the agent
run = project_client.agents.runs.create(thread_id=thread.id, agent_id=agent.id)

while run["status"] in ['queued', 'in_progress', 'requires_action']: # here I use the run as a json object
    run = project_client.agents.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"Run status: {run.status}")
    time.sleep(1)

if run["status"] == "failed": 
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")

if run.status == "failed":
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")

Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.REQUIRES_ACTION
Run status: RunStatus.REQUIRES_ACTION
Run status: RunStatus.REQUIRES_ACTION
Run status: RunStatus.REQUIRES_ACTION
Run status: RunStatus.REQUIRES_ACTION
Run status: RunStatus.REQUIRES_ACTION
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.COMPLETED
CPU times: total: 78.1 ms
Wall time: 18.5 s


# Fetch messages from the thread after the agent run execution

In [7]:
from azure.ai.agents.models import (MessageTextContent, MessageImageFileContent, MessageTextFileCitationAnnotation, \
    MessageTextFilePathAnnotation, MessageTextUrlCitationAnnotation)

image_files = []
annotations = []
citations = []
citation_annotations = []

if run.status == 'completed':
    messages = project_client.agents.messages.list(thread_id=thread.id)
    messages_list = list(project_client.agents.messages.list(thread_id=thread.id))  # Convert iterator to a list
    messages_nr = len(messages_list)
    print(f"Here are the {messages_nr} messages:\n")
    
    for i, message in enumerate(reversed(messages_list), 1):
        j = 0
        print(f"\n===== MESSAGE {i} =====")
        for c in message.content:
            j +=1
            if (type(c) is MessageTextContent):
                print(f"\nMessage {i} / CONTENT {j} (MessageTextContent) --> Text: {c.text.value}")
                for a in c.text.annotations:
                    if type(a) is MessageTextFileCitationAnnotation:
                        print(f">>> Citation in MessageTextContent {j} of message {i}: {a}\n")
                        citations.append(a)
                    elif type(a) is MessageTextFilePathAnnotation:
                        print(f">>> Annotation in MessageTextContent {j} of message {i}: {a}\n")
                        annotations.append(a)
                    elif type(a) is MessageTextUrlCitationAnnotation:
                        print(f">>> Annotation in MessageTextContent {j} of message {i}: {a}\n")
                        citation_annotations.append(a)
                    else:
                        print(f"Unknown type {type(a)}")
            elif (type(c) is MessageImageFileContent):
                print(f"\nMessage {i} / CONTENT {j} (MessageImageFileContent) --> image_file id: {c.image_file.file_id}")
                image_files.append(c.image_file.file_id)
            else:
                print(f"Unknown type {type(a)}")

else:
    print(f"Sorry, I can't proceed because the run status is {run.status}")

Here are the 2 messages:


===== MESSAGE 1 =====

Message 1 / CONTENT 1 (MessageTextContent) --> Text: Com'è il tempo a Palermo domani?

===== MESSAGE 2 =====

Message 2 / CONTENT 1 (MessageTextContent) --> Text: get_weather_agent says: La temperatura a Palermo domani sarà di 7 gradi Celsius e sarà una giornata soleggiata.


# Run Steps

In [8]:
run_steps = list(project_client.agents.run_steps.list(run_id=run.id, thread_id=thread.id))

print(f'Nr of run step(s): {len(run_steps)}\n')
i=0
for rs in run_steps:
    i += 1
    print(f"Run step {i}: {rs}", '\n')

Nr of run step(s): 2

Run step 1: {'id': 'step_LrOihYe3kKnAe7HS9Gj97gBG', 'object': 'thread.run.step', 'created_at': 1752369569, 'run_id': 'run_zkGpfpHhMyiCPiIbSbvvhDn7', 'assistant_id': 'asst_t5umyEWrOCOI2VvuILZ4P73E', 'thread_id': 'thread_AqNTNppu0ifzS3rn5KY1tYX0', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1752369569, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_ADlAGY0tfgF86yJmVAI2qIFL'}}, 'usage': {'prompt_tokens': 167, 'completion_tokens': 29, 'total_tokens': 196, 'prompt_token_details': {'cached_tokens': 0}}} 

Run step 2: {'id': 'step_iETt2xtxniVvjYVMCm1B7CAm', 'object': 'thread.run.step', 'created_at': 1752369556, 'run_id': 'run_zkGpfpHhMyiCPiIbSbvvhDn7', 'assistant_id': 'asst_t5umyEWrOCOI2VvuILZ4P73E', 'thread_id': 'thread_AqNTNppu0ifzS3rn5KY1tYX0', 'type': 'tool_calls', 'status': 'completed', 'cancelled_at': None, 'completed_at': 17

# Collect all resources for this project

In [9]:
all_agents = list_all_agents(client=project_client)
print(all_agents["summary"])

all_threads = list_all_threads(client=project_client)
print(all_threads["summary"])

all_files = list_all_files(client=project_client)
print(all_files["summary"])

all_runs = list_all_runs(client=project_client)
print(all_runs["summary"])

# all_runsteps=list_all_runsteps(project_client)
# print(all_runsteps["summary"])

# all_messages = list_all_messages(project_client)
# print(all_messages["summary"])

all_vectorstores = list_all_vectorstores(client=project_client)
print(all_vectorstores["summary"])

9 agents
10 threads
0 files
9 runs in 10 threads
0 vector stores


# START teardown

In [10]:
# delete all threads

i=0
for thread in all_threads["content"]:
    i += 1
    project_client.agents.threads.delete(thread_id=thread.id)
    print(f"{i} - Thread <{thread.id}> has been deleted")

all_threads = list_all_threads(client=project_client)

print(f"Threads deleted: {i}\n")

1 - Thread <thread_AqNTNppu0ifzS3rn5KY1tYX0> has been deleted
2 - Thread <thread_IAbDkMXbV5r8f10Z9EVZyp9p> has been deleted
3 - Thread <thread_mxBbVjbbsyJOakbR7fJ4G8Kc> has been deleted
4 - Thread <thread_9QdPkHsiN364imCk9pIO7NjQ> has been deleted
5 - Thread <thread_yYtizESsbz5JGXUxArnCYV2H> has been deleted
6 - Thread <thread_igWyNNd5s5oOBAl9CkBscKgz> has been deleted
7 - Thread <thread_0zQ6vGDlqb6R2QgyJARph4IV> has been deleted
8 - Thread <thread_k0YvwFDjlymP95A92BbSCNVi> has been deleted
9 - Thread <thread_WKFhSAGwUCpnqBS330w8IhKT> has been deleted
10 - Thread <thread_esJzT3hSM931UbvpNqflMdAX> has been deleted
Threads deleted: 10



In [11]:
# delete all agents

i=0
for agent in all_agents["content"]:
    i += 1
    project_client.agents.delete_agent(agent_id=agent.id)
    print(f"{i} - Agent <{agent.id}> has been deleted")

all_agents = list_all_agents(client=project_client)

print(f"Agents deleted: {i}\n")

1 - Agent <asst_t5umyEWrOCOI2VvuILZ4P73E> has been deleted
2 - Agent <asst_QDh6tYJoRPjkqgWfFm4XHXcN> has been deleted
3 - Agent <asst_1unNEnOiv31y6qPu7uW8CL59> has been deleted
4 - Agent <asst_jHVTRilLX5SdLieSxjMzvb4Q> has been deleted
5 - Agent <asst_WEuCOvxVs7QOiAu4MikpAD14> has been deleted
6 - Agent <asst_daCh4LYVRtGdRjwqD8hH1Et8> has been deleted
7 - Agent <asst_QAXP4dFLjPbopYZ7JMPJu24s> has been deleted
8 - Agent <asst_AyCgw92s8FgOX2ESC46D59mx> has been deleted
9 - Agent <asst_UoAzDE31AikqvUXKAqmPMkaZ> has been deleted
Agents deleted: 9



# HIC SUNT LEONES